In [1]:
# Loading the table data into the dataframe
df = spark.read.table("Hotel.hotelbook_silver")
display(df.head(5))

StatementMeta(, 37405650-bf99-4b33-8cd1-de538b252299, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ad7243fc-9a91-4fca-9645-cd97017e22cb)

In [2]:
from pyspark.sql.types import*
from delta.tables import*

# Define the Schema for the dimDateGold table
DeltaTable.createIfNotExists(spark) \
     .tableName("Hotel.dimdategold") \
     .addColumn("Date", DateType()) \
     .addColumn("Month", StringType()) \
     .addColumn("Weekday", StringType()) \
     .addColumn("Year", IntegerType()) \
     .addColumn("Season", StringType()) \
     .addColumn("Holiday", StringType()) \
     .execute()


StatementMeta(, 0cfbfba9-cfd6-4a3a-9518-b7e0d9b18489, 4, Finished, Available, Finished)

In [3]:
from pyspark.sql.functions import col, date_add, date_format, year

# Create the dimDateGold dataframe

dfdimDateGold = df.dropDuplicates(["Date", "Month", "Weekday", "Season", "Holiday"]).select(col("Date"), col("Month"), col("Weekday"), col("Season"), col("Holiday"), \
        year("Date").alias("Year"), 
        ).orderBy("Date")
display(dfdimDateGold.head(5))

StatementMeta(, 0cfbfba9-cfd6-4a3a-9518-b7e0d9b18489, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 33e9bc76-0628-4fde-91c1-8f9ed2a72477)

In [4]:
from delta.tables import *

deltatable = DeltaTable.forPath(spark, "Tables/dimdategold")

dfUpdate = dfdimDateGold

deltatable.alias('gold') \
    .merge(
        dfUpdate.alias('updates'),
        'gold.Date = updates.Date'
    ) \
    .whenMatchedUpdate(set={
    
    }) \
    .whenNotMatchedInsert(values={
        "Date": "updates.Date",
        "Month": "updates.Month",
        "Weekday": "updates.Weekday",
        "Year": "updates.Year",
        "Season": "updates.Season",
        "Holiday": "updates.Holiday"
    }) \
    .execute()


StatementMeta(, 0cfbfba9-cfd6-4a3a-9518-b7e0d9b18489, 6, Finished, Available, Finished)

In [10]:
from pyspark.sql.types import *
from delta.tables import *
    
# Create BookingChannel_gold dimension delta table
DeltaTable.createIfNotExists(spark) \
     .tableName("Hotel.dimbookingchannelgold") \
     .addColumn("Booking_Channel", StringType()) \
     .addColumn("Booking_ChannelKey", LongType()) \
     .execute()

StatementMeta(, 0cfbfba9-cfd6-4a3a-9518-b7e0d9b18489, 12, Finished, Available, Finished)

In [11]:
from pyspark.sql.functions import col
# Load data into the df 
dfdimBookingChnlSilver = df.dropDuplicates(["Booking_Channel"]).select(col("Booking_Channel"))
display(dfdimBookingChnlSilver.head(5))

StatementMeta(, 0cfbfba9-cfd6-4a3a-9518-b7e0d9b18489, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a7974a96-8498-489c-8dee-3596cde30a6b)

In [12]:
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit
    
dfdimBooking_temp = spark.read.table("Hotel.dimbookingchannelgold")
    
MAXBookingID = dfdimBooking_temp.select(coalesce(max(col("Booking_ChannelKey")),lit(0)).alias("MAXBookingID")).first()[0]
    
dfdimBookingGold = dfdimBookingChnlSilver.join(dfdimBooking_temp,(dfdimBookingChnlSilver.Booking_Channel == dfdimBooking_temp.Booking_Channel) , "left_anti")
    
dfdimBookingGold = dfdimBookingGold.withColumn("Booking_ChannelKey",monotonically_increasing_id() + MAXBookingID + 1)

# Display the first 10 rows of the dataframe to preview your data

display(dfdimBookingGold.head(10))

StatementMeta(, 0cfbfba9-cfd6-4a3a-9518-b7e0d9b18489, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, adc46d62-ba18-4e51-b212-cb8a12c64003)

In [13]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, 'abfss://edcd96a3-3877-4553-b100-54aeef9c4401@onelake.dfs.fabric.microsoft.com/3f561343-f018-4ab6-a09f-74d0dd933efb/Tables/dimbookingchannelgold')
    
dfUpdates = dfdimBookingGold
    
deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.Booking_Channel = updates.Booking_Channel'
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "Booking_Channel": "updates.Booking_Channel",
      "Booking_ChannelKey": "updates.Booking_ChannelKey"
    }
  ) \
  .execute()

StatementMeta(, 0cfbfba9-cfd6-4a3a-9518-b7e0d9b18489, 15, Finished, Available, Finished)

In [19]:
from pyspark.sql.types import *
from delta.tables import *
    
# Create Guest_gold dimension delta table
DeltaTable.createIfNotExists(spark) \
     .tableName("Hotel.dimguestgold") \
     .addColumn("Guest_Type", StringType()) \
     .addColumn("Guest_Country", StringType()) \
     .addColumn("GuestKey", LongType()) \
     .execute()

StatementMeta(, 0cfbfba9-cfd6-4a3a-9518-b7e0d9b18489, 21, Finished, Available, Finished)

In [20]:
# Loading the data into the dataframe

dfdimGuestSilver = df.dropDuplicates(['Guest_Type' ,'Guest_Country']).select(col("Guest_Type") ,col("Guest_Country"))
display(dfdimGuestSilver.head(10))

StatementMeta(, 0cfbfba9-cfd6-4a3a-9518-b7e0d9b18489, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 08feddf7-220d-414d-ba09-c7185cab81cc)

In [21]:
from pyspark.sql.functions import monotonically_increasing_id, col, lit, max, coalesce
    
dfdimGuest_temp = spark.read.table("Hotel.dimguestgold")
    
MAXGuestID = dfdimGuest_temp.select(coalesce(max(col("GuestKey")),lit(0)).alias("MAXItemID")).first()[0]
    
dfdimGuestGold = dfdimGuestSilver.join(dfdimGuest_temp,(dfdimGuestSilver.Guest_Type == dfdimGuest_temp.Guest_Type) & (dfdimGuestSilver.Guest_Country == dfdimGuest_temp.Guest_Country), "left_anti")
    
dfdimGuestGold = dfdimGuestGold.withColumn("GuestKey",monotonically_increasing_id() + MAXGuestID + 1)
    
# Display the first 10 rows of the dataframe to preview your data

display(dfdimGuestGold.head(10))

StatementMeta(, 0cfbfba9-cfd6-4a3a-9518-b7e0d9b18489, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bcf86391-45f4-43dc-ba0f-7b97831f6370)

In [22]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, 'abfss://edcd96a3-3877-4553-b100-54aeef9c4401@onelake.dfs.fabric.microsoft.com/3f561343-f018-4ab6-a09f-74d0dd933efb/Tables/dimguestgold')
    
dfUpdates = dfdimGuestGold
    
deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.Guest_Type = updates.Guest_Type AND silver.Guest_Country = updates.Guest_Country'
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "Guest_Type": "updates.Guest_Type",
      "Guest_Country": "updates.Guest_Country",
      "GuestKey": "updates.GuestKey"
      
    }
  ) \
  .execute()

StatementMeta(, 0cfbfba9-cfd6-4a3a-9518-b7e0d9b18489, 24, Finished, Available, Finished)

In [2]:
from pyspark.sql.types import *
from delta.tables import *
    
DeltaTable.createIfNotExists(spark) \
    .tableName("Hotel.factreservationsgold") \
    .addColumn("GuestKey", LongType()) \
    .addColumn("Booking_ChannelKey", LongType()) \
    .addColumn("Date", DateType()) \
    .addColumn("Marketing_Spend", IntegerType()) \
    .addColumn("Revenue", FloatType()) \
    .addColumn("Room_Revenue", FloatType()) \
    .addColumn("Occupancy_Rate", FloatType()) \
    .addColumn("ADR", FloatType()) \
    .addColumn("RevPAR", FloatType()) \
    .addColumn("Available_Rooms", IntegerType()) \
    .addColumn("Reserved_Rooms", IntegerType()) \
    .addColumn("Complaints", IntegerType()) \
    .addColumn("Compliment", IntegerType()) \
    .addColumn("Bookings", IntegerType()) \
    .addColumn("No_Shows", IntegerType()) \
    .addColumn("Cancellations", IntegerType()) \
    .addColumn("Market_Segment", FloatType()) \
    .addColumn("Checkouts", IntegerType()) \
    .addColumn("New_Bookings", IntegerType()) \
    .addColumn("Checkins", IntegerType()) \
    .addColumn("Average_Review_Score", FloatType()) \
    .addColumn("Revenue_Managed_Guests", IntegerType()) \
    .addColumn("RevPAR_Managed_Guests", FloatType()) \
    .addColumn("Occupancy_Managed_Guests", IntegerType()) \
    .addColumn("RevPAR_All", FloatType()) \
    .addColumn("Occupancy_All", FloatType()) \
    .addColumn("Room_Revenue_All", FloatType()) \
    .addColumn("Total_Revenue", FloatType()) \
    .addColumn("Operating_Expenses", FloatType()) \
    .addColumn("Fixed_Costs", FloatType()) \
    .addColumn("Variable_Costs", FloatType()) \
    .addColumn("Total_Costs", FloatType()) \
    .addColumn("Profit", FloatType()) \
    .execute()

StatementMeta(, 37405650-bf99-4b33-8cd1-de538b252299, 4, Finished, Available, Finished)

In [3]:
from pyspark.sql.functions import col
    
dfdimBookingChannel_temp = spark.read.table("Hotel.dimbookingchannelgold")
dfdimGuestCountry_temp = spark.read.table("Hotel.dimguestgold")
        
# Create Sales_gold dataframe
    
dffactReservation_gold = df.alias("df1").join(dfdimBookingChannel_temp.alias("df2"),(df.Booking_Channel == dfdimBookingChannel_temp.Booking_Channel), "left") \
        .join(dfdimGuestCountry_temp.alias("df3"),(df.Guest_Type == dfdimGuestCountry_temp.Guest_Type) & (df.Guest_Country == dfdimGuestCountry_temp.Guest_Country), "left") \
    .select(col("df2.Booking_ChannelKey") \
        , col("df3.GuestKey") \
        , col("df1.Date") \
        , col("df1.Marketing_Spend") \
        , col("df1.Revenue") \
        , col("df1.Room_Revenue") \
        , col("df1.Occupancy_Rate") \
        , col("df1.ADR") \
        , col("df1.RevPAR") \
        , col("df1.Available_Rooms") \
        , col("df1.Reserved_Rooms") \
        , col("df1.Complaints") \
        , col("df1.Compliment") \
        , col("df1.Bookings") \
        , col("df1.No_Shows") \
        , col("df1.Cancellations") \
        , col("df1.Market_Segment") \
        , col("df1.Checkouts") \
        , col("df1.New_Bookings") \
        , col("df1.Checkins") \
        , col("df1.Average_Review_Score") \
        , col("df1.Revenue_Managed_Guests") \
        , col("df1.RevPAR_Managed_Guests") \
        , col("df1.Occupancy_Managed_Guests") \
        , col("df1.RevPAR_All") \
        , col("df1.Occupancy_All") \
        , col("df1.Room_Revenue_All") \
        , col("df1.Total_Revenue") \
        , col("df1.Operating_Expenses") \
        , col("df1.Fixed_Costs") \
        , col("df1.Variable_Costs") \
        , col("df1.Total_Costs") \
        , col("df1.Profit")
    ).orderBy(col("df1.Date"), col("df2.Booking_ChannelKey"), col("df3.GuestKey"))
    
# Display the first 10 rows of the dataframe to preview your data
    
display(dffactReservation_gold.head(10))

StatementMeta(, 37405650-bf99-4b33-8cd1-de538b252299, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 18e21319-1da9-46ee-92c7-2b92ba0d90c7)

In [4]:
from delta.tables import *
    
deltaTable = DeltaTable.forPath(spark, 'abfss://edcd96a3-3877-4553-b100-54aeef9c4401@onelake.dfs.fabric.microsoft.com/3f561343-f018-4ab6-a09f-74d0dd933efb/Tables/factreservationsgold')
    
dfUpdates = dffactReservation_gold
    
deltaTable.alias('silver') \
   .merge(
     dfUpdates.alias('updates'),
     'silver.Booking_ChannelKey = updates.Booking_ChannelKey AND silver.GuestKey = updates.GuestKey'
   ) \
    .whenMatchedUpdate(set =
     {
          
     }
   ) \
  .whenNotMatchedInsert(values =
     {
       "Date": "updates.Date",
       "Booking_ChannelKey": "updates.Booking_ChannelKey",
       "GuestKey": "updates.GuestKey",
       "Marketing_Spend": "updates.Marketing_Spend",
       "Revenue": "updates.Revenue",
       "Room_Revenue": "updates.Room_Revenue",
       "Occupancy_Rate": "updates.Occupancy_Rate",
       "ADR": "updates.ADR",
       "RevPAR": "updates.RevPAR",
       "Available_Rooms": "updates.Available_Rooms",
       "Reserved_Rooms": "updates.Reserved_Rooms",
       "Complaints": "updates.Complaints",
       "Compliment": "updates.Compliment",
       "Bookings": "updates.Bookings",
       "No_Shows": "updates.No_Shows",
       "Cancellations": "updates.Cancellations",
       "Market_Segment": "updates.Market_Segment",
       "Checkouts": "updates.Checkouts",
       "New_Bookings": "updates.New_Bookings",
       "Checkins": "updates.Checkins",
       "Average_Review_Score": "updates.Average_Review_Score",
       "Revenue_Managed_Guests": "updates.Revenue_Managed_Guests",
       "RevPAR_Managed_Guests": "updates.RevPAR_Managed_Guests",
       "Occupancy_Managed_Guests": "updates.Occupancy_Managed_Guests",
       "RevPAR_All": "updates.RevPAR_All",
       "Occupancy_All": "updates.Occupancy_All",
       "Room_Revenue_All": "updates.Room_Revenue_All",
       "Total_Revenue": "updates.Total_Revenue",
       "Operating_Expenses": "updates.Operating_Expenses",
       "Fixed_Costs": "updates.Fixed_Costs",
       "Variable_Costs": "updates.Variable_Costs",
       "Total_Costs": "updates.Total_Costs",
       "Profit": "updates.Profit"
     }
   ) \
   .execute()   

StatementMeta(, 37405650-bf99-4b33-8cd1-de538b252299, 6, Finished, Available, Finished)